In [1]:
import requests
import json
import csv 
import pandas as pd
import time

login_url = 'http://localhost:8000/api/user/login/'
headers = {
    "Content-Type": "application/json"
}

# Enter credentials of users
credentials = [] 


df = pd.DataFrame(columns=['subject', 'key', 'DU', 'UD', 'DD', 'UU', 'Overlapped','Number of Errors'])
for userCredentials in credentials : 
    
    UserSession = requests.Session()
    response = UserSession.post(login_url, data=json.dumps(userCredentials), headers=headers)
    if response.status_code == 200 :
        url = "http://localhost:8000/api/user/keystrokes/"
        response = UserSession.get(url, headers=headers)
        KDData = response.json()
        finalData = {}
        PFlag = False
        RFlag = False
        DDFlag = False
        DUFlag = False
        UDFlag = False
        UUFlag = False
        errors = 0
        for keyIndex, keystroke in enumerate(KDData):
            finalData['subject'] = f"s00{keystroke['user']}"
            finalData['key'] = keystroke['key_code']
            if finalData['key'] != 'backspace':
                finalData['Number of Errors'] = errors
            if keystroke['event'] == 'Pressed':
                if finalData['key'] == 'backspace':
                    errors += 1
                    finalData['Number of Errors'] = errors
                for nextKeyIndex,nextKeystroke in enumerate(KDData[keyIndex+1:]):
                    
                    if nextKeystroke['event'] == 'Released' and nextKeystroke['key_code'] == keystroke['key_code']:
                        finalData['DU'] = round((float(nextKeystroke['timestamp']) - float(keystroke['timestamp'])), 5)
                        DUFlag = True
                        if nextKeyIndex == 0:
                            finalData['Overlapped'] = 0
                        else:
                            finalData['Overlapped'] = 1
                    if nextKeystroke['event'] == 'Pressed':
                        finalData['DD'] = round((float(nextKeystroke['timestamp']) - float(keystroke['timestamp'])), 5)
                        DDFlag = True
                    if DUFlag and DDFlag :
                        DDFlag = False
                        DUFlag = False
                        PFlag = True
                        break
            elif keystroke['event'] == 'Released':
                for nextKeyIndex,nextKeystroke in enumerate(KDData[keyIndex+1:]):
                    if nextKeystroke['event'] == 'Pressed' and nextKeystroke['key_code'] != keystroke['key_code']:
                        finalData['UD'] = round((float(nextKeystroke['timestamp']) - float(keystroke['timestamp'])), 5)
                        UDFlag = True
                    if nextKeystroke['event'] == 'Released':
                        finalData['UU'] = round((float(nextKeystroke['timestamp']) - float(keystroke['timestamp'])), 5)
                        UUFlag = True
                    if UDFlag and UUFlag :
                        UDFlag = False
                        UUFlag = False
                        RFlag = True
                        break
            if PFlag and RFlag:
                PFlag = False
                RFlag = False
                df = df.append(finalData, ignore_index=True)
df.to_csv(f"./data/keystrokesDynamics/KeyStrokeDistance.csv", index=False)
display(df)    

,subject,key,DU,UD,DD,UU,Overlapped,Number of Errors
0,s0034,shift,0.33557,1.03070,383.35261,0.09957,1,0
1,s0034,m,0.15325,0.93113,383.35261,1.08439,0,0
2,s0034,backspace,0.15762,0.92795,0.58901,1.08557,0,1
3,s0034,t,0.09175,0.09036,0.23976,0.18212,0,1
4,s0034,o,0.14044,0.14800,0.75439,0.28844,0,1
...,...,...,...,...,...,...,...,...
6531,s0028,backspace,0.11615,0.13424,0.25039,0.13524,0,21
6532,s0028,o,0.00100,0.62184,0.62284,0.67306,0,21
6533,s0028,backspace,0.05122,0.26221,0.10574,0.13933,0,22
6534,s0028,backspace,0.08481,0.12288,0.20769,0.12288,0,23
